In [1]:
import os 
import pandas as pd
import glob

In [2]:
os.chdir('/Users/sivapalansarangan/Word2vec final/Data')

In [3]:
%pwd

'/Users/sivapalansarangan/Word2vec final/Data'

In [4]:
file_extension='.csv'
all_filenames = [i for i in glob.glob(f"*{file_extension}")]

In [5]:
all_filenames[-1]

'மாமலர் - 24.csv'

In [6]:
len(all_filenames)

1941

In [7]:
df = pd.concat([pd.read_csv(f) for f in all_filenames])

In [8]:
os.chdir('/Users/sivapalansarangan/Word2vec final')

In [9]:
df.head()

,Heading,Chapter,Paragraph
0,சொல்வளர்காடு - 1,முதற்காடு : கௌஷீதகம்,தொன்மையான மலைநிலமாகிய கௌஷீதகத்தில் அசிதர் என்ன...
1,NaN,NaN,பாஞ்சாலநாட்டைச் சேர்ந்தவனும் கௌதம கோத்திரத்தவன...
2,NaN,NaN,மாதத்தில் ஒருநாள் தன் மாணவர்களின் தொழில்தேர்ச்...
3,NaN,NaN,அன்று அனல்முகப்பில் அவியிடல் முடிந்தபின் மிச்ச...
4,NaN,NaN,"விழிதிருப்பி அவனை நோக்கிய ஆசிரியர் “சொல், அது ..."


In [15]:
review_text=df["Paragraph"].values

In [16]:
import tensorflow as tf
import re
import string

In [68]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  #lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(input_data,
                                  '[%s]' % re.escape(string.punctuation+u'\u201c'+u'\u201d'+u'\u2018'+u'\u2019'), '')

In [11]:
def clean_text(text ): 
    delete_dict = {sp_character: ' ' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))]) 
    
    return text2.split(' ')

In [17]:
newvec=[clean_text(a) for a in review_text]

In [103]:
#import nltk
#newvec=[nltk.word_tokenize(t) for t in review_text]

In [171]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
# init callback class
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        elif self.epoch % 100 == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        
        
        self.epoch += 1
        self.loss_previous_step = loss

In [175]:
w2v_model = Word2Vec(vector_size = 300,
                     window = 10,
                     min_count = 2,
                     workers = 20,
                     sg = 1,
                     negative = 5,
                     sample = 1e-5)

In [176]:
import time
start = time.time()
w2v_model.build_vocab(newvec)
w2v_model.train(newvec, 
                total_examples=w2v_model.corpus_count, 
                epochs=10, 
                report_delay=1,
                compute_loss = True, # set compute_loss = True
                callbacks=[callback()]) # add the callback class
end = time.time()

Loss after epoch 0: 2520309.75


In [19]:
from gensim.models import Word2Vec

In [20]:
model=Word2Vec(newvec, min_count=2, vector_size=300,sg=1)# sg is o or 1 ie skkipgram or CRow

In [21]:
model.wv['முதற்புலரியில்']

array([-0.08551987,  0.08582722,  0.08568816,  0.22683135,  0.03161886,
       -0.21863335,  0.08204032,  0.18090552,  0.03157439,  0.03097371,
       -0.05046118, -0.06321671,  0.12466056, -0.04982619, -0.01760332,
       -0.00770158,  0.03939118, -0.00860988, -0.02557126, -0.00768459,
       -0.0813006 ,  0.07383581,  0.07249667,  0.0787826 , -0.00838348,
        0.08959188, -0.11618777, -0.05991484, -0.10595969, -0.20409192,
       -0.04750619,  0.04899934,  0.09696218,  0.03751181, -0.03003656,
        0.1384886 , -0.00284336, -0.09611601,  0.02079516,  0.05603093,
       -0.08138581,  0.02675778, -0.17210616, -0.00955326, -0.05270321,
        0.25491297,  0.06388483, -0.0164634 ,  0.08796892,  0.20855558,
        0.12255811, -0.07184163, -0.09571307,  0.11827999, -0.14429389,
        0.09706001, -0.0387624 ,  0.08734038,  0.03265302,  0.10251933,
       -0.03139735, -0.11591085,  0.03599682,  0.13472131,  0.17869356,
        0.0863286 ,  0.00956518,  0.19986445, -0.05610168,  0.13

In [22]:
model.wv.similarity('மாலையில்','காலை')

0.56230605

In [23]:
sims = model.wv.most_similar('காலை', topn=10)
sims

[('மாலை', 0.7423643469810486),
 ('புண்பட்ட', 0.7004303336143494),
 ('புலரியில்', 0.6722503304481506),
 ('விடியற்காலையில்', 0.6453127861022949),
 ('மறுநாள்', 0.6431800127029419),
 ('முதற்கதிர்', 0.6379672884941101),
 ('காலையில்', 0.636972963809967),
 ('முற்புலரியில்', 0.623706042766571),
 ('“நாளை', 0.6153993606567383),
 ('துயின்றேன்', 0.609329104423523)]

In [230]:
model.save("word2vect.model")

In [184]:
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

In [192]:
from gensim.models import KeyedVectors
#wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

In [232]:
model.wv.save_word2vec_format('word2vect.model')
import gensim
from gensim.scripts import word2vec2tensor
gensim.scripts.word2vec2tensor.word2vec2tensor('word2vect.model','word2vec_tensorfile', binary=False)